In [ ]:
%matplotlib inline
import os
import sys
import lsst.daf.persistence as dafPersist
import lsst.afw.table as afwTable
import lsst.afw.geom as afwGeom
import lsst.afw.coord as afwCoord
import lsst.afw.image as afwImage
from lsst.meas.algorithms import LoadIndexedReferenceObjectsTask
from astropy.table import Table, vstack
from astropy import units
import numpy as np
import matplotlib
import matplotlib.pylab as plt

In [ ]:
def get_ref_cat(butler, tract_info, filter_='i'):
    """Extract a sky circle centered on the desired tract from the
    reference catalog.
    """
    ref_config = LoadIndexedReferenceObjectsTask.ConfigClass()
    ref_task = LoadIndexedReferenceObjectsTask(butler, config=ref_config)
    center_coord = tract_info.getCtrCoord()
    radius = max([center_coord.separation(_) for _ in tract_info.getVertexList()])
    ref_cat = ref_task.loadSkyCircle(center_coord, radius, filter_).refCat.asAstropy()
    # Add AB magnitude column for each band.
    for band in 'ugrizy':
        ref_cat.add_column(ref_cat.Column(data=-2.5*np.log10(ref_cat['{}_flux'.format(band)]/3631.), dtype=float),
                           name='{}_mag'.format(band))
    return ref_cat

In [ ]:
def compile_catalog(butler, visits, tract, outfile=None, fluxType='slot_ModelFlux', Flags=None,
                    jc_butler=None):
    if Flags is None:
        Flags = ["base_PixelFlags_flag_saturated", "base_PixelFlags_flag_cr",
                 "base_PixelFlags_flag_interpolated", fluxType + "_flag", "base_SdssCentroid_flag", 
                 "base_SdssCentroid_flag_almostNoSecondDerivative", "base_SdssCentroid_flag_edge",
                 "base_SdssCentroid_flag_noSecondDerivative",
                 "base_SdssCentroid_flag_notAtMaximum", "base_SdssCentroid_flag_resetToPeak", 
                 "base_SdssShape_flag", "base_ClassificationExtendedness_flag"]
    if jc_butler is None:
        jc_butler = butler
    catList = []
    for visit in visits:
        sys.stdout.write('Reading visit %s:' % visit)
        for count, data_ref in enumerate(butler.subset('src', visit=visit)):
            if data_ref.datasetExists():
                dataId = data_ref.dataId
            else:
                continue
            dataId['tract'] = tract
            if not jc_butler.datasetExists('jointcal_wcs', dataId):
                continue
            if count % 10 == 0 :
                sys.stdout.write(' {}'.format(count))
            src = butler.get('src', dataId, immediate=True, flags=afwTable.SOURCE_IO_NO_FOOTPRINTS)
            v = src.asAstropy()
        
            # Get new photometry calibration
            newPhotocal = jc_butler.get('jointcal_photoCalib', dataId, immediate=True)
            newMag = newPhotocal.instFluxToMagnitude(src, fluxType)
            v['newMag'] = newMag[:, 0]
            v['newMagErr'] = newMag[:, 1]

            # select sources
            instFlux = fluxType + '_instFlux'
            instFluxErr = instFlux + 'Err'
            cut = np.ones_like(v['id'], dtype=bool)
            for flag in Flags:
                cut &= v[flag]==False
            cut &= (v[instFlux] > 0) & (v[instFlux]/v[instFluxErr] > 5)
            cut &= v['base_ClassificationExtendedness_value'] < 0.5

            # get calibration object and then magnitudes with errors
            calib = butler.get("calexp_calib", dataId, immediate=True)
            mag, magErr = calib.getMagnitude(v[cut][instFlux], v[cut][instFluxErr])

            cat = v[cut]['id', 'coord_ra', 'coord_dec', instFlux, 'newMag', 'newMagErr']
            cat['mag'] = mag
            cat['magErr'] = magErr

            # add dataId info
            for k in dataId.keys():
                cat[k] = dataId[k]
            
            # Get WCS info from calexp_wcs and from jointcal_wcs
            oldWcs = butler.get("calexp_wcs", dataId, immediate=True)
            newWcs = jc_butler.get('jointcal_wcs', dataId, immediate=True)
        
            newRa = []
            newDec = []
            pixels = oldWcs.skyToPixel([afwGeom.SpherePoint(ra, dec, afwGeom.radians)
                                        for ra, dec in zip(cat['coord_ra'], cat['coord_dec'])])
            newCoord = newWcs.pixelToSky(pixels)                                       
            cat['newRa'] = [float(cd.getRa()) for cd in newCoord]
            cat['newDec'] = [float(cd.getDec()) for cd in newCoord]
            cat['newRa'].unit = units.rad
            cat['newDec'].unit = units.rad

            # hack to get rid of a warning related to a merging conflict
            cat.meta['NOISE_EXPOSURE_ID'] = 0
           
            catList.append(cat)

        sys.stdout.write('\n')
        
    # merge all individual catalogs into a single big one    
    bigCat = vstack(catList)

    if outfile is not None:
        bigCat.write(outfile)
    return bigCat

def plot_tract(tract_info, ax, linestyle=':', color='red', label=None):
    vertex_list = tract_info.getVertexList()
    ra, dec = [], []
    for pos in vertex_list:
        coord = pos.getPosition(afwGeom.radians)
        ra.append(coord.getX())
        dec.append(coord.getY())
    ra.append(ra[0])
    dec.append(dec[0])
    ax.plot(ra, dec, linestyle=linestyle, color=color, label=label)
    return ra, dec

In [ ]:
visits = [204706, 211140, 665807, 682801, 682802, 713246, 731774]
tract = 3633
outfile = 'src_catalog_default_config.fits'
fluxType = "slot_ModelFlux"
butler = dafPersist.Butler("/global/cscratch1/sd/jchiang8/desc/dev/jointcal/output/rerun/jchiang/w_2019_09")
jc_butler = dafPersist.Butler("/global/cscratch1/sd/jchiang8/desc/dev/jointcal/output/rerun/jchiang/w_2019_09_default_configs/")
skyMap = butler.get('deepCoadd_skyMap')

if os.path.isfile(outfile):
    bigCat = Table.read(outfile)
else:
    bigCat = compile_catalog(butler, visits, tract, outfile=outfile, fluxType=fluxType, jc_butler=jc_butler)

In [ ]:
bigCat[:10]

In [ ]:
fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(15, 8))
plot_tract(skyMap[tract], ax0)
ax0.scatter(bigCat['coord_ra'], bigCat['coord_dec'], s=0.01)
ax0.set_xlabel('RA (rad)')
ax0.set_ylabel('Dec (rad)')
ax1.hist(bigCat['mag'], bins=100)
ax1.set_xlabel('i-mag')
plt.savefig('Run2.1i_i-band_7_visits.png');

In [ ]:
plt.savefig('Run2.1i_i-band_7_visits.png');

In [ ]:
from astropy.coordinates import SkyCoord
from astropy import units as u

ref_mag_col = 'i_mag'
ref_cat = get_ref_cat(butler, skyMap[tract])
cut = ref_cat[ref_mag_col] < max(bigCat['mag'])    # Select on magnitude to make the matching easier.
#cut = ref_cat[ref_mag_col] < 26.
ref_cat_coords = SkyCoord(ra=ref_cat[cut]['coord_ra'], dec=ref_cat[cut]['coord_dec'])

bigCat['dist'] = 999999.

dist = []
distNew = []
dMag = []
dMagNew = []
for count, v in enumerate(visits):    
    c = SkyCoord(ra = bigCat[bigCat['visit']==v]['coord_ra'], dec = bigCat[bigCat['visit']==v]['coord_dec'])
    cNew = SkyCoord(ra = bigCat[bigCat['visit']==v]['newRa'], dec = bigCat[bigCat['visit']==v]['newDec'])
    idx, d2d, d3d = c.match_to_catalog_sky(ref_cat_coords)
    idxNew, d2dNew, d3dNew = cNew.match_to_catalog_sky(ref_cat_coords)
    
    dist = dist + d2d.milliarcsecond.tolist()
    distNew = distNew + d2dNew.milliarcsecond.tolist()
    
    dMag = dMag + (bigCat[bigCat['visit']==v]['mag'] - ref_cat[cut][ref_mag_col][idx]).tolist()
    dMagNew = dMagNew + (bigCat[bigCat['visit']==v]['newMag'] - ref_cat[cut][ref_mag_col][idxNew]).tolist()
    
bigCat['dist'] = dist
bigCat['distNew'] = distNew
bigCat['dMag'] = dMag
bigCat['dMagNew'] = dMagNew

In [ ]:
fig = plt.figure(figsize=(8, 6))

magCut = 21
cut = (bigCat['dist'] < 300) & (bigCat['distNew'] < 300) & (bigCat['mag'] < magCut)
label = 'processCcd, median={:.1f} mas'.format(np.median(bigCat[cut]['dist']))
plt.hist(bigCat[cut]['dist'], bins=100, range = [0., 50], histtype='step', label=label)
label = 'jointcal, median={:.1f} mas'.format(np.median(bigCat[cut]['distNew']))
plt.hist(bigCat[cut]['distNew'], bins=100, range = [0., 50], histtype='step', label=label)
plt.title('Astrometric offsets, mag < {}'.format(magCut))

plt.xlabel('Distance to ref. object (mas)', fontsize=15)
plt.legend();

In [ ]:
fig.savefig("Run2.1i_i-band_7_visits_astrometry_constrained.png");

In [ ]:
magCut = 21
cut = (bigCat['dMag'] < 0.08) & (bigCat['dMagNew'] < 0.08) & (bigCat['mag'] < magCut) & (bigCat['dMag'] > -0.08) & (bigCat['dMagNew'] > -0.08)
cut &= (bigCat['distNew'] < 50)

fig = plt.figure(figsize=(8, 6))

label = 'processCcd, med.={:.1f}, sig={:.1f}'.format(np.median(bigCat[cut]['dMag']*1000),
                                                     np.std(bigCat[cut]['dMag']*1000))
plt.hist(bigCat[cut]['dMag']*1000, bins=80, histtype='step', label=label)
label = 'jointcal, med.={:.1f}, sig={:.1f}'.format(np.median(bigCat[cut]['dMagNew']*1000),
                                                   np.std(bigCat[cut]['dMagNew']*1000))
plt.hist(bigCat[cut]['dMagNew']*1000, bins=80, histtype='step', label=label)
plt.xlabel('ref. - meas. (mmag)', fontsize=15)
plt.legend()
plt.title('Photometric offsets, mag < {}'.format(magCut));

In [ ]:
fig.savefig("Run2.1i_i-band_7_visits_photometry_constrained.png");